Xét phương pháp trong Nhóm 2 và Nhóm 3, lựa chọn 1 phân lớp và dựa vào giá trị của hàm quyết định cho
phân lớp đó (ví dụ hàm softmax – logistic hoặc hàm đánh giá score trong SVM) để chuyển bài toán về dạng
hồi quy. Dữ liệu chia train : test như ý (a).
- Thực hiện ít nhất 02 mô hình hồi quy trên tập dữ liệu với đầu ra mới xây dựng.
- Đưa ra kết quả trong trường hợp dữ liệu đầu vào nguyên bản và dữ liệu giảm về còn 1/3 số chiều.
- Đánh giá, so sánh các kết quả thực nghiệm trong mỗi trường hợp. Giải thích xem tại sao lại như vậy.

- Lựa chọn phân lớp Dropout (giá trị xác suất của hàm softmax) để chuyển sang bài toán hồi quy
- Thực hiện với 2 mô hình hồi quy là Linear Regression và KNN

# Transform Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'../data/data_processed/data_processed.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nationality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,-0.294829,0.210069,2.490896,-1.823744,0.350082,-0.386404,-0.145586,0.075111,-0.584526,-0.329669,...,-0.282442,-2.838337,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761,0
1,-0.294829,-0.167406,-0.554068,0.254153,0.350082,-0.386404,-0.145586,-1.254495,-1.218380,-0.829997,...,-0.282442,-0.105726,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199,2
2,-0.294829,-1.111094,2.490896,-1.131112,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.670987,...,-0.282442,-0.105726,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761,0
3,-0.294829,0.210069,0.207173,1.177663,0.350082,-0.386404,-0.145586,1.183116,0.954834,-0.329669,...,-0.282442,-0.105726,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511,2
4,1.356212,0.965018,-0.554068,-1.592866,-2.856470,-0.386404,-0.145586,1.072315,1.045384,0.670987,...,-0.282442,-0.105726,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,-0.294829,-1.111094,3.252137,1.177663,0.350082,-0.386404,-0.145586,-1.254495,-1.399481,-0.329669,...,-0.282442,-0.105726,-0.016033,0.187165,0.467631,-0.199441,1.476924,1.137005,-1.789667,2
4420,-0.294829,-1.111094,0.207173,1.177663,0.350082,-0.386404,10.150427,-1.254495,-1.399481,0.670987,...,-0.282442,-0.105726,-0.522682,-0.808050,0.147747,-0.199441,-0.175007,-0.454253,0.889126,0
4421,-0.294829,-1.111094,-0.554068,0.485030,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.670987,...,-0.282442,0.805144,0.237291,-1.139788,0.627573,-0.199441,0.876222,-1.105222,0.347199,0
4422,-0.294829,-1.111094,-0.554068,-0.207602,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.170659,...,-0.282442,-0.561161,-0.522682,0.187165,0.339678,-0.199441,-0.813253,-1.466871,-1.375511,2


In [3]:
X = df.drop(columns=['Target'])
y = df['Target']

In [4]:
from sklearn.linear_model import LogisticRegression
softmax = LogisticRegression(max_iter=5000)
softmax.fit(X, y)
y_prob = softmax.predict_proba(X)
y_pred = softmax.predict(X)

dff = pd.DataFrame(y_prob, columns=softmax.classes_)
dff

,0,1,2
0,0.519614,0.182403,0.297982
1,0.508375,0.131975,0.359650
2,0.995429,0.004538,0.000033
3,0.122990,0.212526,0.664485
4,0.053636,0.092913,0.853450
...,...,...,...
4419,0.308440,0.254628,0.436933
4420,0.950158,0.047560,0.002282
4421,0.829366,0.138756,0.031879
4422,0.017707,0.045436,0.936857


In [5]:
y_regression_target = y_prob[:, 0]
y_regression_target

array([0.51961426, 0.50837546, 0.995429  , ..., 0.82936551, 0.01770696,
       0.01193866])

In [27]:
y_regression_target = pd.Series(y_regression_target, name='Dropout_Probability')
y_pred_target = pd.Series(y_pred, name='Prediction')

# Thêm cột xác suất và cột dự đoán vào DataFrame gốc
new_df = df.join(y_pred_target)
new_df = new_df.join(y_regression_target)
new_df.head(15)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nationality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target,Prediction,Dropout_Probability
0,-0.294829,0.210069,2.490896,-1.823744,0.350082,-0.386404,-0.145586,0.075111,-0.584526,-0.329669,...,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761,0,0,0.519614
1,-0.294829,-0.167406,-0.554068,0.254153,0.350082,-0.386404,-0.145586,-1.254495,-1.218380,-0.829997,...,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199,2,0,0.508375
2,-0.294829,-1.111094,2.490896,-1.131112,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.670987,...,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761,0,0,0.995429
3,-0.294829,0.210069,0.207173,1.177663,0.350082,-0.386404,-0.145586,1.183116,0.954834,-0.329669,...,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511,2,2,0.122990
4,1.356212,0.965018,-0.554068,-1.592866,-2.856470,-0.386404,-0.145586,1.072315,1.045384,0.670987,...,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199,2,2,0.053636
5,1.356212,0.965018,-0.554068,1.639418,-2.856470,2.389090,-0.145586,1.072315,0.954834,0.670987,...,2.263888,0.187165,0.243712,6.434596,1.739731,-0.671242,-0.406211,2,0,0.349850
6,-0.294829,-1.111094,-0.554068,0.485030,0.350082,-0.386404,-0.145586,0.075111,1.045384,0.170659,...,-0.016033,1.182381,0.789755,-0.199441,1.476924,1.137005,-1.789667,2,2,0.005816
7,-0.294829,0.398806,1.729655,0.254153,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.670987,...,-0.776007,-1.471527,-1.963489,-0.199441,1.476924,1.137005,-1.789667,0,0,0.994225
8,-0.294829,-1.111094,0.968414,0.023276,0.350082,-0.386404,7.862424,-1.254495,-1.399481,0.670987,...,-0.269358,0.518904,0.750957,-0.199441,1.739731,-0.671242,-0.406211,2,2,0.011828
9,-0.294829,-1.111094,-0.554068,0.023276,0.350082,-0.386404,-0.145586,-1.254495,-0.222324,-0.579833,...,1.503914,-0.808050,0.627573,-0.199441,-1.000972,0.124386,1.545607,0,0,0.773876


In [28]:
X

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nationality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,-0.294829,0.210069,2.490896,-1.823744,0.350082,-0.386404,-0.145586,0.075111,-0.584526,-0.329669,...,-0.199273,-0.282442,-2.838337,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
1,-0.294829,-0.167406,-0.554068,0.254153,0.350082,-0.386404,-0.145586,-1.254495,-1.218380,-0.829997,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199
2,-0.294829,-1.111094,2.490896,-1.131112,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.670987,...,-0.199273,-0.282442,-0.105726,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
3,-0.294829,0.210069,0.207173,1.177663,0.350082,-0.386404,-0.145586,1.183116,0.954834,-0.329669,...,-0.199273,-0.282442,-0.105726,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511
4,1.356212,0.965018,-0.554068,-1.592866,-2.856470,-0.386404,-0.145586,1.072315,1.045384,0.670987,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,-0.294829,-1.111094,3.252137,1.177663,0.350082,-0.386404,-0.145586,-1.254495,-1.399481,-0.329669,...,-0.199273,-0.282442,-0.105726,-0.016033,0.187165,0.467631,-0.199441,1.476924,1.137005,-1.789667
4420,-0.294829,-1.111094,0.207173,1.177663,0.350082,-0.386404,10.150427,-1.254495,-1.399481,0.670987,...,-0.199273,-0.282442,-0.105726,-0.522682,-0.808050,0.147747,-0.199441,-0.175007,-0.454253,0.889126
4421,-0.294829,-1.111094,-0.554068,0.485030,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.670987,...,-0.199273,-0.282442,0.805144,0.237291,-1.139788,0.627573,-0.199441,0.876222,-1.105222,0.347199
4422,-0.294829,-1.111094,-0.554068,-0.207602,0.350082,-0.386404,-0.145586,1.072315,0.954834,0.170659,...,-0.199273,-0.282442,-0.561161,-0.522682,0.187165,0.339678,-0.199441,-0.813253,-1.466871,-1.375511


In [30]:
y = y_regression_target
y

0       0.519614
1       0.508375
2       0.995429
3       0.122990
4       0.053636
          ...   
4419    0.308440
4420    0.950158
4421    0.829366
4422    0.017707
4423    0.011939
Name: Dropout_Probability, Length: 4424, dtype: float64

# Regression

In [68]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

In [69]:
# giảm chiều còn 1/3 chiều
from sklearn.decomposition import PCA
pca = PCA(n_components=int(X.shape[1] / 3))
X_reduced = pca.fit_transform(X)

In [70]:
splits = [0.2, 0.4, 0.3]

# Ham để đánh giá mô hình
def evaluate_model(X, y, model, label='Gốc'):
    results = []
    for split in splits:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        results.append({
            'Loại dữ liệu': label,
            'Tỷ lệ train:test': f'{int(100*(1-split))}:{int(100*split)}',
            'MSE': round(mse, 4),
            'R^2': round(r2, 4),
            'MAE': round(mae, 4)
        })
    return results


# Linear Regression

In [55]:
linear_reg = LinearRegression()

## Dữ liệu gốc

In [67]:
lr_results_original = evaluate_model(X, y, linear_reg, label='Dữ liệu gốc')
pd.DataFrame(lr_results_original)

,Loại dữ liệu,Tỷ lệ train:test,MSE,R^2,MAE
0,Dữ liệu gốc,80:20,0.0087,0.9345,0.0697
1,Dữ liệu gốc,60:40,0.0081,0.9375,0.0674
2,Dữ liệu gốc,70:30,0.0081,0.9377,0.0672


## Dữ liệu giảm chiều 1/3

In [66]:
lr_results_reduced = evaluate_model(X_reduced, y,linear_reg, label='Dữ liệu giảm chiều')
pd.DataFrame(lr_results_reduced)

,Loại dữ liệu,Tỷ lệ train:test,MSE,R^2,MAE
0,Dữ liệu giảm chiều,80:20,0.0299,0.7751,0.1308
1,Dữ liệu giảm chiều,60:40,0.0292,0.7732,0.1308
2,Dữ liệu giảm chiều,70:30,0.0288,0.7786,0.1298


- So sánh hiệu suất giữa dữ liệu gốc và dữ liệu đã giảm chiều của mô hình hồi quy tuyến tính:
    + Dữ liệu gốc có MSE nhỏ hơn đáng kể so với MSE của dữ liệu đã giảm chiều, chứng tỏ mô hình dự đoán tốt hơn trên dữ liệu gốc.
    + Dữ lệu gốc có R2 cao hơn (~93%) so dữ liệu đã giảm chiều (~77%), mô thấy mô hình giải thích được phần lớn phương sai trong dữ liệu gốc.
    + MAE(sai số tuyệt đối trung bình) của dữ liệu giảm chiều chỉ bằng 1 nửa MAE của dữ liệu gốc.
  -> Giảm chiều làm mất mát thông tin quan trọng trong dữ liệu, dẫn đến hiệu suất mô hình kém hơn rõ rệt.

- So sánh giữa các tỷ lệ train - test:
    + Cả hai loại dữ liệu đều cho kết quả khá ổn định giữa các tỷ lệ train:test.
    + Tuy nhiên, dữ liệu gốc vẫn luôn giữ hiệu suất tốt hơn ở mọi tỷ lệ chia.


# KNN

In [80]:
knn_reg = KNeighborsRegressor(n_neighbors=10)

## Dữ liệu gốc

In [81]:
knn_results_original = evaluate_model(X, y, knn_reg, label='Dữ liệu gốc')
pd.DataFrame(knn_results_original)

,Loại dữ liệu,Tỷ lệ train:test,MSE,R^2,MAE
0,Dữ liệu gốc,80:20,0.0262,0.8033,0.1024
1,Dữ liệu gốc,60:40,0.0266,0.7936,0.1038
2,Dữ liệu gốc,70:30,0.0254,0.8048,0.1016


## Dữ liệu giảm chiều

In [82]:
knn_results_reduced = evaluate_model(X_reduced, y, knn_reg, label='Dữ liệu giảm chiều')
pd.DataFrame(knn_results_reduced)

,Loại dữ liệu,Tỷ lệ train:test,MSE,R^2,MAE
0,Dữ liệu giảm chiều,80:20,0.0256,0.8081,0.1055
1,Dữ liệu giảm chiều,60:40,0.0255,0.8024,0.1053
2,Dữ liệu giảm chiều,70:30,0.0254,0.8050,0.1053


- So sánh hiệu suất giữa dữ liệu gốc và dữ liệu giảm chiều của mô hình hồi quy KNN:
    + Hai loại dữ liệu có MSE rất tương đương nhau, chênh lệch không đáng kể.
    + Dữ liệu giảm chiều có R2 nhỉnh hơn, cho thấy mô hình giải thích được phương sai tốt hơn một chút sau khi giảm chiều.
    + MAE của dữ liệu gốc nhỏ hơn một chút, cho thấy dự đoán gần giá trị thực hơn một chút.
  -> Giảm chiều không gây giảm hiệu suất của mô hình KNN, thậm chí có giúp cải thiện R2 một chút, điều này là hợp lý, vì KNN hoạt động dựa trên khoảng cách, nên việc giảm chiều có thể giúp loại bỏ nhiễu

- So sánh giữa các tỷ lệ train test:
  + Cả hai loại dữ liệu đều cho kết quả ổn định giữa các tỷ lệ train:test (sai số và R² không biến động nhiều).
  + Đây là điểm mạnh của KNN — không quá nhạy với thay đổi nhỏ về dữ liệu huấn luyện

# So sánh kết quả mô hình Linear Regression và KNN

1. Trên dữ liệu gốc:
- Mô hình hồi quy tuyến tính cho kết quả vượt trội hơn rõ rệt với MSE ~ 0.0081, R2 ~ 0.937 và MAE ~ 0.067, chứng tỏ khả năng dự đoán tốt và phù hợp với phân bố tuyến tính của dữ liệu.
- Mô hình KNN đạt MSE ~ 0.0254, R2 ~ 0.804 và MAE ~ 0.102, thấp hơn đáng kể về hiệu suất so với hồi quy tuyến tính. Điều này cho thấy KNN không khai thác được tốt các mối quan hệ tuyến tính trong không gian gốc.

2. Trên dữ liệu đã giảm chiều:
- Kết quả giữa hai mô hình có xu hướng đảo ngược nhẹ. KNN Regression cho kết quả tốt hơn hoặc tương đương với Linear Regression về R2 (KNN: ~0.808 so với Linear Regression: ~0.775) và có MSE nhỏ hơn đáng kể (Linear Regression ~0.029 so với KNN ~0.0255).
- Như vậy, việc giảm chiều đã gây mất mát thông tin ảnh hưởng tiêu cực tới hồi quy tuyến tính, trong khi lại cải thiện nhẹ hiệu suất KNN, do mô hình này hưởng lợi từ không gian đặc trưng gọn hơn và ít nhiễu hơn.

3. Kết luận
- Linear Regression phù hợp hơn với dữ liệu gốc có cấu trúc tuyến tính rõ ràng, và hoạt động rất tốt trong điều kiện không bị giảm chiều.
- KNN Regression không đạt hiệu suất cao với dữ liệu gốc, nhưng ổn định và hiệu quả hơn sau khi giảm chiều, đặc biệt trong bối cảnh dữ liệu có thể chứa nhiễu hoặc chiều dư thừa.
- Việc chọn mô hình nên dựa trên đặc điểm phân bố và cấu trúc của dữ liệu, cũng như mục tiêu cân bằng giữa hiệu suất và tính đơn giản.